## Data prep

In [8]:
import pandas as pd
df = pd.read_csv("data/songdata.csv")
df.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [49]:
vocab = set()
for song in df["text"]:
    chars = set(song)
    vocab = vocab.union(chars)
vocab_size = len(vocab)
print("Vocab size:", len(vocab))

Vocab size: 76


In [50]:
chars = list(vocab)
char2idx = { char:i for i,char in enumerate(chars) }
idx2char = { i:char for i,char in enumerate(chars) }

In [51]:
def build_samples(song, buffer_length):
    tokens = song

    x_train = []
    y_train = []
    for i in range(0, len(song)):
        if i+buffer_length+1 >= len(tokens):
            continue
            
        x_train.append(tokens[i:i+buffer_length])
        y_train.append(tokens[i+buffer_length])

    return x_train,y_train

In [52]:
x_train, y_train = [], []
for song in df["text"][:100]:
    xs, ys = build_samples(song, 6)
    x_train.extend(xs)
    y_train.extend(ys)
print("Training data length:", len(x_train))
print("X[0]:", x_train[0])
print("Y[0]:", y_train[0])

Training data length: 133204
X[0]: Look a
Y[0]: t


## Train/Test split

In [53]:
from sklearn.model_selection import train_test_split
import math

# 80% Train, 10% Dev, 10% Test
X_train, X_test, Y_train, Y_test = train_test_split(x_train, y_train, test_size=0.2)
X_dev, X_test, Y_dev, Y_test = train_test_split(X_test, Y_test, test_size=0.5)

In [64]:
import numpy as np
from random import shuffle
import util

SEQUENCE_LENGTH = 6

def generate_batches(data_length, mini_batch_size):
    for begin in range(0, data_length, mini_batch_size):
        end = min(begin + mini_batch_size, data_length)
        yield begin, end

def load_batch(xs, ys, begin, end):
    batch_size = end-begin
    
    x_train = np.zeros((batch_size, SEQUENCE_LENGTH, vocab_size))
    y_train = np.zeros((batch_size, vocab_size))
    
    xs_batch = xs[begin:end]
    ys_batch = ys[begin:end]
    
    c = list(zip(xs_batch, ys_batch))
    shuffle(c)
    xs_batch, ys_batch = zip(*c)
    
    for i in range(batch_size):
        x_train[i] = util.one_hot_encode_sequence(xs_batch[i], char2idx)
        y_train[i] = util.one_hot_encode(ys_batch[i], char2idx)
    
    return x_train, y_train

In [71]:
batches = generate_batches(len(X_train), 512)
begin, end = next(batches)
x_batch, y_batch = load_batch(X_train, Y_train, begin, end)
print(x_batch.shape, y_batch.shape)

(512, 6, 76) (512, 76)


In [84]:
for i in range(10):
    xs = ''.join(util.one_hot_decode_sequence(x_batch[i], idx2char))
    y = util.one_hot_decode(y_batch[i], idx2char)
    print(f"{i} '{xs}' -> {y}")

0 'give m' -> e
1 ' chest' ->  
2 'w, I'm' ->  
3 ' move ' -> o
4 'rust i' -> n
5 'aking ' -> t
6 'f you ' ->  
7 ' I do,' ->  
8 'mean" ' ->  
9 'y die ' ->  


## Training time!

In [88]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, Dropout, GRU
from keras.layers import LeakyReLU

def build_model(vocab_size):
    model = Sequential()
    model.add(LSTM(512, input_shape=(SEQUENCE_LENGTH, vocab_size)))
    model.add(Dropout(0.5))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Dense(vocab_size, activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer="adam", metrics = ['accuracy'])
    return model

model = build_model(vocab_size)
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 512)               1206272   
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 76)                38988     
Total params: 1,245,260
Trainable params: 1,245,260
Non-trainable params: 0
_________________________________________________________________


In [91]:
batches = generate_batches(len(X_train), 4096)

In [92]:
for begin, end in batches:
    x_batch, y_batch = load_batch(X_train, Y_train, begin, end)
    model.fit(x_batch, y_batch, batch_size=256, epochs=4)

Epoch 1/4
4096/4096 [==============================] - 1s 181us/step - loss: 1.9627 - accuracy: 0.4438
Epoch 2/4
4096/4096 [==============================] - 1s 185us/step - loss: 1.8818 - accuracy: 0.4651
Epoch 3/4
4096/4096 [==============================] - 1s 187us/step - loss: 1.8321 - accuracy: 0.4734
Epoch 4/4
4096/4096 [==============================] - 1s 190us/step - loss: 1.7827 - accuracy: 0.4888
Epoch 1/4
4096/4096 [==============================] - 1s 187us/step - loss: 1.9097 - accuracy: 0.4514
Epoch 2/4
4096/4096 [==============================] - 1s 194us/step - loss: 1.8290 - accuracy: 0.4780
Epoch 3/4
4096/4096 [==============================] - 1s 189us/step - loss: 1.7937 - accuracy: 0.4832
Epoch 4/4
4096/4096 [==============================] - 1s 192us/step - loss: 1.7277 - accuracy: 0.5034
Epoch 1/4
4096/4096 [==============================] - 1s 197us/step - loss: 1.8586 - accuracy: 0.4790
Epoch 2/4
4096/4096 [==============================] - 1s 191us/step - lo

4096/4096 [==============================] - 1s 191us/step - loss: 1.2717 - accuracy: 0.6191
Epoch 1/4
4096/4096 [==============================] - 1s 193us/step - loss: 1.7447 - accuracy: 0.5083
Epoch 2/4
4096/4096 [==============================] - 1s 190us/step - loss: 1.5472 - accuracy: 0.5498
Epoch 3/4
4096/4096 [==============================] - 1s 200us/step - loss: 1.4328 - accuracy: 0.5845
Epoch 4/4
4096/4096 [==============================] - 1s 201us/step - loss: 1.3352 - accuracy: 0.6111
Epoch 1/4
4096/4096 [==============================] - 1s 212us/step - loss: 1.6136 - accuracy: 0.5305
Epoch 2/4
4096/4096 [==============================] - 1s 187us/step - loss: 1.4496 - accuracy: 0.5696
Epoch 3/4
4096/4096 [==============================] - 1s 185us/step - loss: 1.3470 - accuracy: 0.6025
Epoch 4/4
4096/4096 [==============================] - 1s 177us/step - loss: 1.2420 - accuracy: 0.6309
Epoch 1/4
4096/4096 [==============================] - 1s 177us/step - loss: 1.6401

In [93]:
model.save_weights("weights_char_rnn.h5")